In [1]:
from bs4 import BeautifulSoup
import re
import urllib
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import xlwt

### 抓去给定网页的

In [2]:
def askURL(url):
    # 设置为浏览器Header的User-Agent
    header = {
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15'
    }
    
    # 发送请求
    request = urllib.request.Request(url, headers = header)
    try:
        # 取得响应
        response = urllib.request.urlopen(request)
        # 网页整体内容
        html = response.read()
        print("%s crawl success" % url)
    except urllib.error.URLError as e:
        print("%s crawl fail" % url)
        if hasattr(e, 'code'):
            print(e.code)
        if hasattr(e, 'reason'):
            print(e.reason)
    return html

In [3]:
#获取相关内容
def getData(baseurl):
    findLink=re.compile(r'<a href="(.*?)">')#找到影片详情链接
    findImgSrc=re.compile(r'<img.*src="(.*?)"',re.S)#找到影片图片
    findTitle=re.compile(r'<span class="title">(.*)</span>')#找到片名
    #找到评分
    findRating=re.compile(r'<span class="rating_num" property="v:average">(.*)</span>')
    #找到评价人数
    findJudge=re.compile(r'<span>(\d*)人评价</span>')
    #找到概况
    findInq=re.compile(r'<span class="inq">(.*)</span>')
    #找到影片相关内容：导演，主演，年份，地区，类别
    findBd=re.compile(r'<p class="">(.*?)</p>',re.S)
    #去掉无关内容
    remove=re.compile(r' |\n|</br>|\.*')
    datalist=[]
    for i in range(0,10):
        url=baseurl+str(i*25)
        # print(url)
        html=askURL(url)
        soup = BeautifulSoup(html, "html.parser")
        for item in soup.find_all('div',class_='item'):#找到每一个影片项
            data=[]
            item=str(item)#转换成字符串
            # 影片详情链接
            link=re.findall(findLink,item)[0]
            data.append(link)#添加详情链接  
            imgSrc=re.findall(findImgSrc,item)[0]
            data.append(imgSrc)#添加图片链接
            titles=re.findall(findTitle,item)
            #片名可能只有一个中文名，没有外国名
            if(len(titles)==2):
                ctitle=titles[0]
                data.append(ctitle)#添加中文片名
                otitle=titles[1].replace("/","")#去掉无关符号
                data.append(otitle)#添加外国片名
            else:
                data.append(titles[0])#添加中文片名
                data.append(' ')#留空
                
            rating=re.findall(findRating,item)[0]
            data.append(rating)#添加评分
            judgeNum=re.findall(findJudge,item)[0]
            data.append(judgeNum)#添加评论人数
            inq=re.findall(findInq,item)
            #可能没有概况
            if len(inq)!=0:
                inq=inq[0].replace("。","")#去掉句号
                data.append(inq)#添加概况
            else:
                data.append(' ')#留空
            bd=re.findall(findBd,item)[0]
            bd=re.sub(remove,"",bd)
            bd=re.sub('<br(\s+)?\/?>(\s+)?'," ",bd) #去掉<br >
            bd=re.sub('/', " ",bd)#替换/
            data.append(bd.strip())
            datalist.append(data)
    return datalist


In [11]:
#将相关数据写入excel中
def saveData(datalist,savepath):
    col=['电影详情链接','图片链接','影片中文名','影片外国名',
                '评分','评价数','概况','相关信息']
    
    df = pd.DataFrame(datalist, columns=col)
    df.to_excel(savepath, sheet_name="豆瓣电影")
    """
    book=xlwt.Workbook(encoding='utf-8',style_compression=0)
    sheet=book.add_sheet('豆瓣电影',cell_overwrite_ok=True)
    col=('电影详情链接','图片链接','影片中文名','影片外国名',
                '评分','评价数','概况','相关信息')
    for i in range(0,8):
        sheet.write(0,i,col[i])#列名
    for i in range(0,250):
        data=datalist[i]
        for j in range(0,8):
            sheet.write(i+1,j,data[j])#数据
    book.save(savepath)#保存
    """
    
    return df

In [ ]:
print ("开始爬取......")
baseurl='https://movie.douban.com/top250?start='
datalist=getData(baseurl)
print("爬取完成......")

savapath=u'豆瓣电影.xlsx'
df = saveData(datalist,savapath)

# df = pd.read_excel(savapath)
print(df.head())
print(df.info())
matplotlib.rcParams['font.size']=20
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.scatter(df['评分'],range(1,251))
plt.xlabel('score')
plt.ylabel('rank')

#集中趋势的直方图
plt.subplot(1,2,2)
plt.hist(df['评分'],bins=15)
plt.xlabel('score')
plt.ylabel('number')


开始爬取......
https://movie.douban.com/top250?start=0 crawl success
https://movie.douban.com/top250?start=25 crawl success
https://movie.douban.com/top250?start=50 crawl success
https://movie.douban.com/top250?start=75 crawl success
https://movie.douban.com/top250?start=100 crawl success
https://movie.douban.com/top250?start=125 crawl success
https://movie.douban.com/top250?start=150 crawl success
https://movie.douban.com/top250?start=175 crawl success
